# Case Study 
- detect Object on Sample Image <br>
<img src="sample2.jpg" style="height:300px">
<img src="sample3.jpg" style="height:300px">

- Ekspektasi akhir bisa mengukur jarak" berikut <br><br>
<img src="resource/expected.jpeg" style="width:500px">

- temukan `lower` dan `upper` untuk warna yang ingin kita threshold
    - gunakan `color_range_picker.py`
    - untuk menggunakannya, jalankan pada terminal raspberry pi,
        ```
        python3 color_range_picker.py --mode 1 --camera 0
        ```
        atau, jika ingin dari gambar, 
        ```
        python3 color_range_picker.py --mode 2 --image sample1.jpg
        ```

- Result Thresholding di `sample2.jpg`
    - blue lower `[96, 83, 89]`
    - blue upper `[117, 255, 255]`
    - object lower `[0, 7, 62]`
    - object upper `[180, 30, 137]` <br><br>
    <img src="resource/sample2_1_threshold.png" style="width:400px"></img>
    <img src="resource/sample2_2_threshold.png" style="width:400px"></img>

- Deteksi object sangat sulit karena pengaruh noise.
- kita gunakan gambar yang *background nya sudah dihilangkan* (`sample2-removebg.png`) untuk mempermudah deteksi.
<img src="sample2-removebg.png" style="width:400px"></img>

- Result Thresholding di `sample2-removebg.png`
    - object lower `[0, 3, 13]`
    - object upper `[180, 61, 125]` <br><br>
    <img src="resource/sample2_3_threshold.png" style="width:400px"></img>

In [1]:
import cv2
import numpy as np

- Test Draw Contour blue & object

In [2]:
# define range of blue color in HSV
# lower = np.array([96, 83, 89])
# upper = np.array([117, 255, 255])

# define range of object color in HSV
lower = np.array([0, 7, 62])
upper = np.array([180, 30, 137])


img = cv2.imread('sample2-removebg.png')
f = 320 / img.shape[1]
img = cv2.resize(img, (0,0), fx=f, fy=f)

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv.copy(), lower, upper)
res = cv2.bitwise_and(img, img, mask= mask)

# find contour from mask image using RETR_EXTERNAL method
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to the original image
for contour_item in contours:
    cv2.drawContours(img, contour_item, -1, (0, 0, 255), 2)

# show image
cv2.imshow('Draw Contour',img)
cv2.imshow('Threshold Image', res)
cv2.waitKey(0) # display the window infinitely until any keypress
cv2.destroyAllWindows()

- Result Find Contour in `sample2-removebg.png` <br><br>
<img src="resource/sample2-1-contour.png" style="width:500px"></img>
<img src="resource/sample2-4-contour.png" style="width:500px"></img>
<br><br>

- Result Find Contour in `sample2-removebg-edited.png` <br><br>
<img src="resource/sample2-2-contour.png" style="width:500px"></img>
<img src="resource/sample2-3-contour.png" style="width:500px"></img>

- removing small contour using `aspecRation` and `extent`

In [4]:
# define range of blue color in HSV
# lower = np.array([96, 83, 89])
# upper = np.array([117, 255, 255])

# define range of object color in HSV
lower = np.array([0, 7, 62])
upper = np.array([180, 30, 137])


img = cv2.imread('sample2-removebg-edited.png')
f = 320 / img.shape[1]
img = cv2.resize(img, (0,0), fx=f, fy=f)

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv.copy(), lower, upper)
res = cv2.bitwise_and(img, img, mask= mask)


# find contour from mask image using RETR_EXTERNAL method
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to the original image
# write bounding rectangle at position x,y
for contour_item in contours:
    
    x, y, w, h = cv2.boundingRect(contour_item)
    area = cv2.contourArea(contour_item)
    
    # calculate aspect_ratio & extent
    aspect_ratio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area

    
    # filter contour with aspect ratio and extend blue
    #if aspect_ratio < 3 and aspect_ratio > 1 and extent > 0.2 and  extent < 0.4 and area > 1500:
    
    # filter contour with aspect ratio and extend object
    if aspect_ratio < 3 and aspect_ratio > 1 and extent > 0.1 and  extent < 0.3 and area > 700:    
        cv2.drawContours(img, contour_item, -1, (0,255,255), 1)
        
        text = "(%d, %d, %d, %d)" % (x, y, w, h)
        cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1)

        cv2.rectangle(img,        # input image
            (x, y),               # (x1, y1)
            (x + w, y + h),       # (x2, y2)
            (0,0,255),            # (B, G, R)
            2)                    # thickness
        print ("aspect ratio : %.2f, extent : %.2f, area : %.2f" % (aspect_ratio, extent, area))

# show image
cv2.imshow('Draw Contour',img)
cv2.imshow('Threshold Image', res)
cv2.waitKey(0) # display the window infinitely until any keypress
cv2.destroyAllWindows()        

aspect ratio : 1.47, extent : 0.22, area : 972.50
aspect ratio : 1.50, extent : 0.18, area : 984.50


- Result Filter Contour in `sample2-removebg.png` <br><br>
<img src="resource/sample2-1-filter.png" style="width:500px"></img>
<img src="resource/sample2-2-filter.png" style="width:500px"></img>
<br><br>

- Result Find Contour in `sample2-removebg-edited.png` <br><br>
<img src="resource/sample2-1-filter.png" style="width:500px"></img>
<img src="resource/sample2-4-filter.png" style="width:500px"></img>
<br><br>

- Kombinasikan program deteksi warna biru dan object
- Agar hasil deteksinya bagus kita gunakan `drawer.py` untuk membantu men-drawing result deteksi ke gambar.

In [2]:
import drawer 

dw = drawer.Drawer()

In [3]:
# define range of object color in HSV
lower = np.array([0, 7, 62])
upper = np.array([180, 30, 137])


img = cv2.imread('sample2-removebg-edited.png')
f = 320 / img.shape[1]
img = cv2.resize(img, (0,0), fx=f, fy=f)

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv.copy(), lower, upper)
res = cv2.bitwise_and(img, img, mask= mask)


# find contour from mask image using RETR_EXTERNAL method
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to the original image
# write bounding rectangle at position x,y
for contour_item in contours:
    
    x, y, w, h = cv2.boundingRect(contour_item)
    area = cv2.contourArea(contour_item)
    
    # calculate aspect_ratio & extent
    aspect_ratio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area

    
    # filter contour with aspect ratio and extend object
    if aspect_ratio < 3 and aspect_ratio > 1 and extent > 0.1 and  extent < 0.3 and area > 700:    

        # draw line and put text via drawer.py draw_distance
        img = dw.draw_distance(img, 0, y+h, x, y+h)

# draw line and put text via drawer.py draw cross line 
img = dw.draw_cross_line(img)

# show image
cv2.imshow('Draw Contour',img)
cv2.imshow('Threshold Image', res)
cv2.waitKey(0) # display the window infinitely until any keypress
cv2.destroyAllWindows()        

- Result Kombinasi program dengan `drawer.py` di `sample2-removebg-edited.png` <br><br>
<img src="resource/sample2-1-results.png" style="width:900px"></img>